In [15]:
# optinal
import os
os.environ["https_proxy"] = "http://192.168.1.12:7891"

# Get data with markdown heading struture

In [17]:
import re
import json
from datasets import load_dataset
import sys
sys.path.append("../..")
# from utils.sys_prompts import IT_SYS_PROMPT as SYSTEM_PROMPT
# from utils.sys_prompts import IT_SYS_PROMPT_deepseek_concise as SYSTEM_PROMPT
from utils.sys_prompts import SYS_PROMPT_formatter_deepseek_concise_2 as SYSTEM_PROMPT
import random

INSTRUCTION_RESPONSE_FORMAT = """\
<instruction>
{instruction}
</instruction>
<response>
{response}
</response>
"""

def get_markdown_structure_data(url, split="train", output_file="dataset.json"):
    dataset = load_dataset(url, split=split)
    # since we want to emphasize the logical hierarchy
    def check_structure_markdown(text):
        structure_markdown_regrex = r"^#{1,6} .*$"
        res = re.findall(structure_markdown_regrex, text["output"], re.MULTILINE)
        if len(res) > 0:
            return True
        else:
            return False
    def check_length(text):
        length = len(text["output"])
        if  (length > 50) and (length < 2000):
            return True
        else:
            return False
    dataset = dataset.filter(
        check_structure_markdown,
    )
    dataset = dataset.filter(
        check_length,
    )
    print("\033[92m Number of data after filtering: \033[0m", len(dataset))
    
    def formatting_prompts_func(examples):
        instruction = examples["instruction"]
        input       = examples["input"]
        output      = examples["output"]
        return {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                # few-shot
                # {'role': 'user', 'content': INSTRUCTION_RESPONSE_FORMAT.format(instruction=(fewshot_example["instruction"]+fewshot_example["input"]).strip(), response=fewshot_example['output'])},
                # {'role': 'assistant', 'content': fewshot_response},
                {'role': 'user', 'content': INSTRUCTION_RESPONSE_FORMAT.format(instruction=(instruction+input).strip(), response=output)}
            ],
            # 'answer': extract_hash_answer(output)
        }
    
    dataset = dataset.map(formatting_prompts_func, batched=False)
    dataset = dataset.remove_columns([col for col in dataset.column_names if col != "prompt"])
    # dataset = dataset.shuffle(seed=42).select(range(min(1000, len(dataset))))  # Ensure we get up to 1000 samples
    print("\033[92m Save number of data: \033[0m", len(dataset))
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(dataset.to_list(), f, indent=4, ensure_ascii=False)
    
    return dataset


In [18]:
dataset = get_markdown_structure_data("yahma/alpaca-cleaned", "train", "alpaca-markdown.json")

 Number of data after filtering:  339
 Save number of data:  339


In [9]:
print(dataset[0]['prompt'][1]['content'])

<instruction>
Refactor this code and add comments.def factorialize(num):
  factorial = 1
  for i in range(1, num):
    factorial *= i
  
  return factorial
</instruction>
<response>
```
# This is a function that calculates the factorial of a given number
def factorialize(num):
  # Set the variable 'factorial' to 1
  factorial = 1
  # Iterate from 1 to 'num' using the range function
  for i in range(1, num):
    # Multiply the 'factorial' variable by the current number in the iteration
    factorial *= i
  # Return the final value of 'factorial'
  return factorial
```
</response>



In [4]:
print(dataset[0]['prompt'][0]['content'])

You are a meticulous organizational assistant specialized in structuring instruction-response pairs into a standardized markdown format. Please carefully process the input according to the following specifications:

---
tags:
  - {general_tag} {general_tag}/{sub_tag}
---
# Instruction
[The original instruction text]

# Summary
[A brief yet comprehensive summary of the response]

## Details
[The original response content]

Here are Processing Guidelines:
- The `tags` section consists of pairs of general tags and sub tags in the following format:  `{general_tag} {general_tag}/{sub_tag}`. For example: `environment environment/renewable_energy`.
- Keep the heaidng levels in the original response and adjust heading levels as needed to maintain proper hierarchy and avoid jumping heading levels.

The instructions and responses are enclosed within `<instruction>` and `<response>` XML tags, respectively. Please process the following instruction-response pairs with precision and attention to str

# Get data without markdown heading level struture

In [10]:
import re
import json
from datasets import load_dataset
import sys
sys.path.append("../..")
from utils.sys_prompts import IT_SYS_PROMPT as SYSTEM_PROMPT
import random

INSTRUCTION_RESPONSE_FORMAT = """\
<instruction>
{instruction}
</instruction>
<response>
{response}
</response>
"""

def get_naive_data(url, split="train", output_file="dataset.json", number_data=7000):
    dataset = load_dataset(url, split=split)
    # since we want to emphasize the logical hierarchy
    def check_structure_markdown(text):
        structure_markdown_regrex = r"^#{1,6} .*$"
        res = re.findall(structure_markdown_regrex, text["output"], re.MULTILINE)
        if len(res) > 0:
            return True
        else:
            return False
    def check_length(text):
        length = len(text["output"])
        if  (length > 50) and (length < 2000):
            return True
        else:
            return False
    dataset = dataset.filter(lambda x: (not check_structure_markdown(x)) and check_length(x))
    print("\033[92m Number of data after filtering: \033[0m", len(dataset))
    
    def formatting_prompts_func(examples):
        instruction = examples["instruction"]
        input       = examples["input"]
        output      = examples["output"]
        return {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                # few-shot
                # {'role': 'user', 'content': INSTRUCTION_RESPONSE_FORMAT.format(instruction=(fewshot_example["instruction"]+fewshot_example["input"]).strip(), response=fewshot_example['output'])},
                # {'role': 'assistant', 'content': fewshot_response},
                {'role': 'user', 'content': INSTRUCTION_RESPONSE_FORMAT.format(instruction=(instruction+input).strip(), response=output)}
            ],
            # 'answer': extract_hash_answer(output)
        }
    
    dataset = dataset.map(formatting_prompts_func, batched=False)
    dataset = dataset.remove_columns([col for col in dataset.column_names if col != "prompt"])
    dataset = dataset.shuffle(seed=42).select(range(min(number_data, len(dataset))))  # Ensure we get up to 1000 samples
    print("\033[92m Save number of data: \033[0m", len(dataset))
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(dataset.to_list(), f, indent=4, ensure_ascii=False)
    
    return dataset


In [11]:
dataset = get_naive_data("yahma/alpaca-cleaned", "train", "alpaca-naive.json", 5000)

 Number of data after filtering:  42776
 Save number of data:  5000


In [12]:
text = "You are a meticulous organizational assistant specialized in structuring instruction-response pairs into a standardized markdown format. Please carefully process the input according to the following specifications:\n\n---\ntags:\n  - {broad_category} {broad_category}/{specific_topic}\n---\n# Instruction\n[The original instruction text]\n\n# Summary\n[A brief yet comprehensive summary of the response]\n\n## Details\n[The full response content]\n• Adjust heading levels as needed to maintain proper hierarchy and avoid jumping heading levels.\n\nThe instructions and responses are enclosed within `<instruction>` and `<response>` XML tags, respectively. Please process the following instruction-response pairs with precision and attention to structural integrity.\n"
print(text)

You are a meticulous organizational assistant specialized in structuring instruction-response pairs into a standardized markdown format. Please carefully process the input according to the following specifications:

---
tags:
  - {broad_category} {broad_category}/{specific_topic}
---
# Instruction
[The original instruction text]

# Summary
[A brief yet comprehensive summary of the response]

## Details
[The full response content]
• Adjust heading levels as needed to maintain proper hierarchy and avoid jumping heading levels.

The instructions and responses are enclosed within `<instruction>` and `<response>` XML tags, respectively. Please process the following instruction-response pairs with precision and attention to structural integrity.

